In [1]:
BASE_PATH = "/home/diegodp/Documents/PhD/Paper_3/SolO_SDO_EUI/"

from sys import path

path.append(f"{BASE_PATH}Scripts/")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import makedirs

from datetime import datetime, timedelta
import PyEMD
from EMDComparison.signalHelpers import Signal, SignalFunctions, compareTS
from astropy.convolution import Box1DKernel, convolve
from sunpy.time import parse_time
import idlsave


In [2]:
# Set the unsafe, target safe, and dataFolder
unsafe_dir = "/home/diegodp/Documents/PhD/Paper_3/SolO_SDO_EUI/unsafe/"
saveFolder = f"{unsafe_dir}ISSI/"
dataFolder = f"/home/diegodp/Documents/PhD/Paper_3/SolO_SDO_EUI/Scripts/ISSI/data/"
makedirs(saveFolder, exist_ok=True)

# IN SITU DATA
df_is = pd.read_csv(f"{dataFolder}small_ch_in_situ.csv")
df_is.index = pd.to_datetime(df_is["Time"])
del df_is["Time"]

# REMOTE DATA
rs_171 = idlsave.read(f'{dataFolder}small_ch_171_lc_in.sav', verbose=False)
rs_193 = idlsave.read(f'{dataFolder}small_ch_193_lc_in.sav', verbose=False)
ch_flux = idlsave.read(f'{dataFolder}chflux.sav', verbose=False)

# 171 and 193 observations
time_array = rs_171.date_obs_171.copy()
time_array = [t.decode() for t in list(time_array)]

df_171 = pd.DataFrame(
    {
        'plume': rs_171.lc_171_plume_in,
        'cbpoint': rs_171.lc_171_bp_in,
        'chplume': rs_171.lc_171_ch_plume_in,
        'chole': rs_171.lc_171_ch_in,
        'qsun': rs_171.lc_171_qs_in,
    },
    index=pd.to_datetime(time_array))

df_193 = pd.DataFrame(
    {
        'plume': rs_193.lc_193_plume_in,
        'cbpoint': rs_193.lc_193_bp_in,
        'chplume': rs_193.lc_193_ch_plume_in,
        'chole': rs_193.lc_193_ch_in,
        'qsun': rs_193.lc_193_qs_in,
    },
    index=pd.to_datetime(time_array))

# Open and Bright point flux
flux_time = ch_flux.hmitimes.copy()
flux_time = [t.decode() for t in list(flux_time)]

df_flux = pd.DataFrame(
    {
        "ch_open_flux": ch_flux.chofluxes,
        "ch_bpoint_flux": ch_flux.chbpfluxes,
    },
    index=pd.to_datetime(flux_time, format="%Y.%m.%d_%H:%M:%S_TAI"))

# Times of relevance

In [7]:
df_flux

,ch_open_flux,ch_bpoint_flux
2018-10-28 00:00:00,-1.308001e+19,6.376789e+19
2018-10-28 00:12:00,-1.330703e+19,6.847705e+19
2018-10-28 00:24:00,-1.253655e+19,8.740412e+19
2018-10-28 00:36:00,-1.258985e+19,1.023376e+20
2018-10-28 00:48:00,-1.205261e+19,8.876795e+19
...,...,...
2018-10-30 07:00:00,-1.329372e+19,9.233607e+19
2018-10-30 07:12:00,-1.291377e+19,1.103461e+20
2018-10-30 07:24:00,-1.206358e+19,1.040168e+20
2018-10-30 07:36:00,-1.196758e+19,1.134233e+20


In [3]:
# We set a large possible set of periodicities
PeriodMinMax = [5, 60]

# Selecting ~ 10 hours of in situ observations
timeInsitu = (datetime(2018, 10, 31, 12), datetime(2018, 10, 31, 22))
timeInsitu = (datetime(2018, 10, 31, 12), datetime(2018, 10, 31, 15))

# Selecting all of remote observations
time171 = (df_171.head(1).index.to_pydatetime()[0],
           df_171.tail(1).index.to_pydatetime()[0])
time193 = (df_193.head(1).index.to_pydatetime()[0],
           df_193.tail(1).index.to_pydatetime()[0])

# Selecting?
timeFlux = (parse_time("2018-10-28 00:00:00.000").datetime,
            parse_time("2018-10-30 07:48:01.000").datetime)

dfRemotes = {
    "df_171": df_171[time171[0]:time171[1]],
    "df_193": df_171[time193[0]:time193[1]],
    "df_flux": df_flux[timeFlux[0]:timeFlux[1]],
}

# Using compareTS from signalFunctions

In [4]:
def AIA_compare(
                AIA=dfRemotes["df_171"], 
                PSP=df_is,
                AIA_id="171"):
    AIA.columns = [f"{AIA_id}_{i}" for i in AIA.columns]
    PSP.columns = [f"PSP_{i}" for i in PSP.columns]
    specificFolder = f"{saveFolder}{AIA_id}/"
    compareTS(
        dfSelf=AIA,
        dfOther=PSP,
        cadSelf=60,
        cadOther=60,
        labelOther="PSP",
        winDispList=[60],
        corrThrPlotList=np.arange(0.65, 1, 0.05),
        PeriodMinMax=PeriodMinMax,
        filterPeriods=True,
        savePath=specificFolder,
        useRealTime=True,
        expectedLocationList=[],
        detrend_box_width=200,
        delete=True,
        showFig=True,
        renormalize=False,
        showSpeed=True,
        SPCKernelName="psp",
    )


AIA_compare(AIA=dfRemotes["df_171"], PSP=df_is, AIA_id="171")

Normalising PSP_Br
Normalising 171_plume
Saving IMF files to /home/diegodp/Documents/PhD/Paper_3/SolO_SDO_EUI/unsafe/ISSI/171/171_plume/PSP_Br/60s/5 - 60/
Found 980 displacements possible.


KeyboardInterrupt: 